# Export to GeoJSON with Indicator Attributes

This Jupyter Notebook will walk you through the process of converting your DevInfo database into the possible output formats of:
- GeoJSON (only spatial information and a reference area id)
- GeoJSON (full attribute AND spatial information)
- *TODO ::: Add link to Notebook for Exporting from GeoJSON (this) to FileGeodatabase*

### **Important**

You need to have previously run the Export to Shapefiles notebook **before** running this notebook as it will need a folder location of the individual shapefiles. 

## Import the needed python libraries
install the GDAL pythyon library (ogr import below) by opening the anaconda prompt and using `conda install gdal`

In [1]:
import os
import sys
import json
import pyodbc
from osgeo import ogr
from pathlib import Path

## Set defaults

In [16]:
# location of access database
access_database = r'Z:\dev\nepal-import\NepalInfo2016_for_python.accdb'

# where is your working folder? usually just the name of the country you are working on
output_base = 'nepal'

# location of individual shapefiles for each layer. 
# this is created by running the Export to Shapefiles jupyter notebook, see *Important* note above
output_shapes_folder = 'shapes'

# where do you want to write the output geojson files?
output_geojson_folder = 'geojson_full'

# flag for full properties & geometries
# True for full attributes and geometry
# False for only REF_AREA and REF_AREA_ID and geometry
full_props = False

## Check for the `Esri Shapefile` driver
The DevInfo Access database stores the geometry as a Shapefile. We can use this driver to read that into our script

In [3]:
shp_driver_lbl = 'Esri Shapefile'
shp_driver = ogr.GetDriverByName(shp_driver_lbl)
if shp_driver is None:
    print ('{} driver not available.'.format(shp_driver_lbl))
else:
    print ('{} driver IS available.'.format(shp_driver_lbl))

Esri Shapefile driver IS available.


## Prepare to query the DevInfo Access Database
Here we will map our DevInfo fields to the DSD as defined here. **TODO :: add link(s) to reference data schema**

This is our ouptut data schema that will show in the GeoJSON `properties`

In [4]:
# 'DSD_FIELD' : 'DevInfoField'
field_mappings = {
    'INDICATOR_ID' : 'Indicator_NId',
    'INDICATOR' : 'Indicator_Name',
    'REF_AREA' : 'Area_Name',
    'REF_AREA_ID' : 'Area_ID',
    'OBS_VALUE' : 'Data_Value',
    'UNIT_ID' : 'Unit_NId',
    'UNIT' : 'Unit_Name',
    'TIME_PERIOD' : 'TimePeriod'
}

## Query the DevInfo Access Database

In [5]:
connStr = (
    r'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};'
    r'DBQ={};'.format(access_database)
)

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT UT_Data.Indicator_NId, UT_Indicator_en.Indicator_Name, UT_Data.Data_Value, UT_Unit_en.Unit_NId, UT_Unit_en.Unit_Name, UT_Area_en.Area_ID, UT_Area_en.Area_Name, UT_Area_Level_en.Area_Level_Name, UT_Area_en.Area_Level, UT_Indicator_Classifications_en.IC_Name, UT_Indicator_Classifications_en.Publisher, UT_TimePeriod.TimePeriod, UT_Subgroup_Vals_en.Subgroup_Val, UT_Subgroup_Type_en.Subgroup_Type_Name, UT_Area_Map_Layer.Layer_NId
FROM ((((UT_Area_Map_Layer INNER JOIN ((UT_Area_Level_en INNER JOIN (UT_Subgroup_Vals_en INNER JOIN (UT_Unit_en INNER JOIN (UT_Indicator_en INNER JOIN (UT_Indicator_Unit_Subgroup INNER JOIN (UT_TimePeriod INNER JOIN (UT_Indicator_Classifications_en INNER JOIN (UT_Area_en INNER JOIN UT_Data ON UT_Area_en.[Area_NId] = UT_Data.[Area_NId]) ON UT_Indicator_Classifications_en.IC_NId = UT_Data.Source_NId) ON UT_TimePeriod.TimePeriod_NId = UT_Data.TimePeriod_NId) ON UT_Indicator_Unit_Subgroup.IUSNId = UT_Data.IUSNId) ON UT_Indicator_en.Indicator_NId = UT_Indicator_Unit_Subgroup.Indicator_NId) ON UT_Unit_en.Unit_NId = UT_Indicator_Unit_Subgroup.Unit_NId) ON UT_Subgroup_Vals_en.Subgroup_Val_NId = UT_Indicator_Unit_Subgroup.Subgroup_Val_NId) ON UT_Area_Level_en.Area_Level = UT_Area_en.Area_Level) INNER JOIN UT_Area_Map ON UT_Area_en.Area_NId = UT_Area_Map.Area_NId) ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map.Layer_NId) INNER JOIN UT_Area_Map_Metadata_en ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map_Metadata_en.Layer_NId) INNER JOIN UT_Subgroup_Vals_Subgroup ON UT_Subgroup_Vals_en.Subgroup_Val_NId = UT_Subgroup_Vals_Subgroup.Subgroup_Val_NId) INNER JOIN UT_Subgroup_en ON UT_Subgroup_Vals_Subgroup.Subgroup_NId = UT_Subgroup_en.Subgroup_NId) INNER JOIN UT_Subgroup_Type_en ON UT_Subgroup_en.Subgroup_Type = UT_Subgroup_Type_en.Subgroup_Type_NId
ORDER BY UT_Data.Indicator_NId
"""

crsr = cnxn.execute(sql)

rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

sucessfully executed data query :: 124272 rows returned


## Chunk up the data
Chunk up the data by Indicator. This will let us create one layer per Indicator

In [6]:
chunks = {}
for row in rows:
    
    ind_id = row.Indicator_NId
    layer_id = row.Layer_NId
    
    if ind_id not in chunks:
        chunks[ind_id] = {}
    
    if layer_id not in chunks[ind_id]:
        chunks[ind_id][layer_id] = {}
        chunks[ind_id][layer_id]['rows'] = []
    
    chunks[ind_id][layer_id]['rows'].append(row)

print ('done chunking data by indicator, by layer')

done chunking data by indicator, by layer


## Join the Attribute Data & Spatial Data
Finally, we will step through our data to create individual geojson files for each layer, for each layer.

In [17]:
# store the already reference geometry in memory for faster recall
geom_cache = {}

for c in chunks:
    # test with just one indicator
    if c != 4:
        continue
    
    ind = c
    lyrs = chunks[c]

    for lyr in lyrs:
        feature_collection = {
            'type' : 'FeatureCollection',
            'features': []
        }
    
        rows = lyrs[lyr]['rows']

        for row in rows:
            layer_id = row.Layer_NId
            area_id = row.Area_ID
            
            # test with just one layer
#             if layer_id != 52:
#                 continue
            
            geom = None
            if full_props:
                if area_id not in geom_cache:
                    # look for the already created shapefile
                    shp_file_path = '{}/{}.shp'.format(os.path.join(output_base, output_shapes_folder), layer_id)

                    # check to see if we were able to get the shapefile
                    # TODO: add logging rather than just printing an exception
                    shp_file = shp_driver.Open(shp_file_path)
                    if shp_file is None:
    #                     print ('{} shape file not found'.format(shp_file_path))
                        continue

                    layer = shp_file.GetLayer()
                    
                    where_clause = 'ID_ = \'{}\''.format(area_id)
                    layer.SetAttributeFilter(where_clause)
                    feature = layer.GetNextFeature()
                    if feature is None:
                        print ('unable to get feature for layer {} :: where {}'.format(layer_id, where_clause))
                        continue
                        
                    geom_ref = feature.GetGeometryRef()

                    geom_json_str = geom_ref.ExportToJson()

                    geom = json.loads(geom_json_str)

                    #store in cache
                    geom_cache[area_id] = geom

                    # clean up
                    del feature
                    del layer
                    del shp_file

                else:
                    geom = geom_cache[area_id]
    
            # setup the new feature
            feature = {
                'properties': {},
                'geometry': geom
            }
            
            # if we are going to include full attributes and geometry
            if full_props:
                for field in field_mappings:
                    feature['properties'][field] = getattr(row, field_mappings[field])
            else:
                feature['properties']['REF_AREA'] = getattr(row, field_mappings['REF_AREA'])
                feature['properties']['REF_AREA_ID'] = getattr(row, field_mappings['REF_AREA_ID'])

            feature_collection['features'].append(feature)
        
        # create filename for output geojson file
        new_ind = str(c)
        new_ind_id = new_ind.replace('.', '_')
        layer_name = 'indicator_{}_layer_{}.geojson'.format(new_ind_id, layer_id)

        # full path for the output geojson file
        full_path = Path(os.path.join(output_base, output_geojson_folder, layer_name))    
    
        print ('writing {} features to {}'.format(len(feature_collection['features']), layer_name))
        with open(full_path, 'w') as file:
            file.write(json.dumps(feature_collection))

del geom_cache
print ('done')

writing 15 features to indicator_4_0_layer_50.geojson
writing 4 features to indicator_4_0_layer_51.geojson
writing 5 features to indicator_4_0_layer_52.geojson
writing 4 features to indicator_4_0_layer_48.geojson
done
